In [1]:
import pandas as pd
import numpy as np
import random
import re
import nltk
from pprint import pprint
from nltk.corpus import stopwords
from nltk import word_tokenize
import pymorphy3
from string import punctuation
from string import digits
from tqdm import tqdm
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from pymystem3 import Mystem
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn import metrics
import time, psutil, os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
sns.set_theme()
import scipy
from scipy import sparse
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from gensim.models import KeyedVectors
russian_stopwords = stopwords.words("russian")

In [2]:
new_words=('фото', 'splash', 'news/all', 'over', 'press', 'globallookpress', 'photographer')

In [3]:
random.seed(2020)

In [4]:
df = pd.read_csv('sport_and_nauka_and_showbiz_full_dataset_1095_shuffle_new.csv')

In [5]:
df

,URL,Text,Title,Class
0,https://7days.ru/news/ego-razvedut-na-bablo-bi...,"Диме Билану декабре исполнится 40 лет, он до с...",«Его разведут на бабло»: Билана после свадьбы ...,ШОУ-БИЗНЕС СЛУХИ
1,https://argumenti.ru/showbiz/2020/04/660300,Известная актриса Светлана Ходченкова поделила...,Ходченкова покорила поклонников в элегантном н...,ШОУ-БИЗНЕС
2,https://7days.ru/news/brosil-zhenu-kutsenko-po...,Гоша Куценко накануне посетил премьеру фильма ...,Бросил жену? Куценко появился на публике с бер...,ШОУ-БИЗНЕС СЛУХИ
3,https://7days.ru/news/o-sostoyanii-docheri-uit...,"Дочь Уитни Хьюстон, Бобби Кристина Браун была ...",О состоянии дочери Уитни Хьюстон поступают про...,ШОУ-БИЗНЕС СЛУХИ
4,https://argumenti.ru/science/2017/01/518254,"Важное открытие сделали ученые, представляющие...","Ученые рассказали, когда человек начинает стареть",НАУКА СЛУХИ
...,...,...,...,...
6565,https://7days.ru/news/zakon-on-ne-narushal-pri...,Даня Милохин около недели назад предпринял поп...,«Закон он не нарушал»: приемный отец рассказал...,ШОУ-БИЗНЕС СЛУХИ
6566,https://argumenti.ru/science/2016/08/463230,Позитивный настрой просто хорошее настроение ч...,Позитивный подход усиливает эффективность спор...,НАУКА СЛУХИ
6567,https://argumenti.ru/science/2019/12/643864,Ученые из Гронингенского университета провели ...,Ученые предрекли Европе сильное похолодание в ...,НАУКА СЛУХИ
6568,https://www.sports.ru/football/1110514050-rubi...,Защитник Монтассар Тальби покинул «Рубин» прод...,«Рубин» продал Тальби «Лорьяну». Защитник сбор...,СПОРТ


In [6]:
encoder = LabelEncoder()

In [7]:
df['ClassEncoded'] = encoder.fit_transform(df['Class'])

In [8]:
df['Text'] = df['Text'].str.slice(0,1350)

In [9]:
df['Text'] = df['Text'].apply(lambda x: ' '.join(x.split(' ')[:-1]))

In [10]:
morph = pymorphy3.MorphAnalyzer()

In [11]:
def process(text):
    return list(t.lower() for t in nltk.word_tokenize(text) if t.lower() not in new_words)

In [12]:
data = [process(t) for t in df['Text'][:]]

In [13]:
data[0]

['диме',
 'билану',
 'декабре',
 'исполнится',
 '40',
 'лет',
 ',',
 'он',
 'до',
 'сих',
 'пор',
 'ни',
 'разу',
 'не',
 'был',
 'женат',
 '.',
 'сначала',
 'ему',
 'было',
 'не',
 'до',
 'этого',
 'певец',
 'строил',
 'карьеру',
 'был',
 'сосредоточен',
 'на',
 'работе',
 '.',
 'сейчас',
 'же',
 'жизни',
 'артиста',
 ',',
 'как',
 'говорят',
 ',',
 'многое',
 'изменилось',
 '.',
 'он',
 'якобы',
 'живет',
 'некой',
 'аленой',
 'усачевой',
 ',',
 'одной',
 'из',
 'его',
 'фанаток',
 '.',
 'говорят',
 ',',
 'что',
 'билан',
 'даже',
 'собирается',
 'сделать',
 'ей',
 'предложение',
 ',',
 'однако',
 'против',
 'свадьбы',
 'выступает',
 'яна',
 'рудковская',
 '.',
 'светская',
 'львица',
 'считает',
 ',',
 'что',
 'женитьба',
 'может',
 'негативно',
 'сказать',
 'на',
 'творческих',
 'успехах',
 'билана',
 '.',
 'кроме',
 'того',
 ',',
 'она',
 'подозревает',
 ',',
 'что',
 'певца',
 'могут',
 'использовать',
 '.',
 'не',
 'секрет',
 ',',
 'что',
 'дима',
 'богатый',
 'человек',
 'его',

In [14]:
def lemmatizer(texts):
    return [[morph.parse(word)[0] for word in text] for text in texts]

In [15]:
morph_data = lemmatizer(data)

In [16]:
morph_data[0]

[Parse(word='диме', tag=OpencorporaTag('NOUN,anim,masc,Name sing,datv'), normal_form='дима', score=0.4, methods_stack=((DictionaryAnalyzer(), 'диме', 631, 2),)),
 Parse(word='билану', tag=OpencorporaTag('NOUN,anim,masc,Name sing,datv'), normal_form='билан', score=1.0, methods_stack=((DictionaryAnalyzer(), 'билану', 27, 2),)),
 Parse(word='декабре', tag=OpencorporaTag('NOUN,inan,masc sing,loct'), normal_form='декабрь', score=1.0, methods_stack=((DictionaryAnalyzer(), 'декабре', 195, 5),)),
 Parse(word='исполнится', tag=OpencorporaTag('VERB,perf,intr sing,3per,futr,indc'), normal_form='исполниться', score=1.0, methods_stack=((DictionaryAnalyzer(), 'исполнится', 652, 9),)),
 Parse(word='40', tag=OpencorporaTag('NUMB,intg'), normal_form='40', score=1.0, methods_stack=((NumberAnalyzer(score=0.9), '40'),)),
 Parse(word='лет', tag=OpencorporaTag('NOUN,inan,masc plur,gent'), normal_form='год', score=0.998492, methods_stack=((DictionaryAnalyzer(), 'лет', 1221, 11),)),
 Parse(word=',', tag=Openc

In [17]:
def extract_lemma(texts):
    norm = []
    for t in texts:
        res = []
        for word in t:
            n = word.normal_form
            res.append(n)
        norm.append(res)
    return norm

In [18]:
data_norm = extract_lemma(morph_data)

In [19]:
data_norm[0]

['дима',
 'билан',
 'декабрь',
 'исполниться',
 '40',
 'год',
 ',',
 'он',
 'до',
 'сей',
 'пора',
 'ни',
 'раз',
 'не',
 'быть',
 'женатый',
 '.',
 'сначала',
 'он',
 'быть',
 'не',
 'до',
 'это',
 'певец',
 'строить',
 'карьера',
 'быть',
 'сосредоточить',
 'на',
 'работа',
 '.',
 'сейчас',
 'же',
 'жизнь',
 'артист',
 ',',
 'как',
 'говорить',
 ',',
 'многое',
 'измениться',
 '.',
 'он',
 'якобы',
 'жить',
 'некий',
 'алёна',
 'усачёв',
 ',',
 'один',
 'из',
 'он',
 'фанатка',
 '.',
 'говорить',
 ',',
 'что',
 'билан',
 'даже',
 'собираться',
 'сделать',
 'она',
 'предложение',
 ',',
 'однако',
 'против',
 'свадьба',
 'выступать',
 'ян',
 'рудковский',
 '.',
 'светский',
 'львица',
 'считать',
 ',',
 'что',
 'женитьба',
 'мочь',
 'негативно',
 'сказать',
 'на',
 'творческий',
 'успех',
 'билан',
 '.',
 'кроме',
 'тот',
 ',',
 'она',
 'подозревать',
 ',',
 'что',
 'певец',
 'мочь',
 'использовать',
 '.',
 'не',
 'секрет',
 ',',
 'что',
 'дима',
 'богатый',
 'человек',
 'он',
 'деньга

In [20]:
data_norm = [w for w in data_norm if not w in new_words]

In [21]:
data_norm[0]

['дима',
 'билан',
 'декабрь',
 'исполниться',
 '40',
 'год',
 ',',
 'он',
 'до',
 'сей',
 'пора',
 'ни',
 'раз',
 'не',
 'быть',
 'женатый',
 '.',
 'сначала',
 'он',
 'быть',
 'не',
 'до',
 'это',
 'певец',
 'строить',
 'карьера',
 'быть',
 'сосредоточить',
 'на',
 'работа',
 '.',
 'сейчас',
 'же',
 'жизнь',
 'артист',
 ',',
 'как',
 'говорить',
 ',',
 'многое',
 'измениться',
 '.',
 'он',
 'якобы',
 'жить',
 'некий',
 'алёна',
 'усачёв',
 ',',
 'один',
 'из',
 'он',
 'фанатка',
 '.',
 'говорить',
 ',',
 'что',
 'билан',
 'даже',
 'собираться',
 'сделать',
 'она',
 'предложение',
 ',',
 'однако',
 'против',
 'свадьба',
 'выступать',
 'ян',
 'рудковский',
 '.',
 'светский',
 'львица',
 'считать',
 ',',
 'что',
 'женитьба',
 'мочь',
 'негативно',
 'сказать',
 'на',
 'творческий',
 'успех',
 'билан',
 '.',
 'кроме',
 'тот',
 ',',
 'она',
 'подозревать',
 ',',
 'что',
 'певец',
 'мочь',
 'использовать',
 '.',
 'не',
 'секрет',
 ',',
 'что',
 'дима',
 'богатый',
 'человек',
 'он',
 'деньга

In [22]:
X_train, X_test, y_train, y_test = train_test_split(data_norm, df['ClassEncoded'], random_state = 2020, test_size = 0.2)

In [23]:
#ytrain_encoded = tf.keras.utils.to_categorical(y_train, num_classes=6, dtype = 'int32')
#ytest_encoded = tf.keras.utils.to_categorical(y_test, num_classes=6, dtype = 'int32')

In [24]:
#X_train = [str (item) for item in X_train]

In [25]:
#X_test = [str (item) for item in X_test]

In [26]:
#TfidfVec = TfidfVectorizer(ngram_range = (1, 1))
#X_train_tfidf = TfidfVec.fit_transform(X_train)
#X_test_tfidf = TfidfVec.transform(X_test)

In [27]:
le = LabelEncoder()
y_train_number = le.fit_transform(y_train)
y_test_number = le.fit_transform(y_test)

In [28]:
y_train_number

array([1, 4, 5, ..., 0, 0, 5], dtype=int64)

In [29]:
dict_train = {'tokens': X_train, 'label': y_train_number} 

In [30]:
data_train = pd.DataFrame(dict_train)

In [31]:
dict_test = {'tokens': X_test, 'label': y_test_number} 

In [32]:
data_test = pd.DataFrame(dict_test)

In [33]:
#import zipfile
#import wget
#model_url = 'http://vectors.nlpl.eu/repository/11/184.zip'
#m = wget.download(model_url)
#model_file = model_url.split('/')[-1]
#with zipfile.ZipFile(model_file, 'r') as archive:
#    stream = archive.open('model.bin')
#    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [34]:
model = Word2Vec(X_train, window=2, negative=15, vector_size=300, epochs=10, sg=1)

In [35]:
model.wv.most_similar('город', topn=10)

[('индия', 0.7211979627609253),
 ('черноморский', 0.7123941779136658),
 ('рим', 0.7079176902770996),
 ('восточный', 0.7078136801719666),
 ('побережье', 0.7038128972053528),
 ('юго-восточный', 0.7029277086257935),
 ('парк', 0.7000864148139954),
 ('эмират', 0.698810338973999),
 ('гостиница', 0.6976697444915771),
 ('соседний', 0.6963327527046204)]

In [36]:
vocab=list(model.wv.key_to_index.keys())
len(vocab)

9413

In [37]:
model.wv.save_word2vec_format(fname='word2vecsg_window2')